### Example 01

Shows how to create a simple agent with:
* checkpointing
* integration with OCI APM tracing
* usage of AgentBase class

In [1]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

# for OCI APM integration
# we need this only to start the trace, no explicit code in Step1 and Step2
from py_zipkin.zipkin import zipkin_span
from py_zipkin import Encoding

# opur classes
from oci_langgraph import AgentBase, APMTransport, OracleCheckpointSaver

# configurations from config files
from config import ENABLE_TRACING, APM_BASE_URL

# from here we get connection params (user, pwd...) 
from config_private import CONNECT_ARGS, APM_PUBLIC_KEY

In [2]:
# the DB table where we store checkpoints
TABLE_NAME = "EXAMPLE01_CHECKPOINTS"

AGENT_NAME = "Example01"

In [3]:
# configure the checkpointing
# the first time it creates the DB table
checkpointer = OracleCheckpointSaver(connect_args=CONNECT_ARGS, table_name=TABLE_NAME)

# configure the transport for APM integration
transporter = APMTransport(base_url=APM_BASE_URL,
                          public_key=APM_PUBLIC_KEY,
                          enable_tracing=ENABLE_TRACING)

#### Create a workflow with two steps

In [4]:
#
# They only do print
#

# AgentBase implements runnable and has the invoke() method.
# we need to define the handle_invoke()

class Step1(AgentBase):
    """ first step """
    def handle_invoke(self, input, config=None, **kwargs):
        print("Executed step1...")


class Step2(AgentBase):
    """ second step """
    def handle_invoke(self, input, config=None, **kwargs):
        print("Executed step1...")

step1 = Step1(AGENT_NAME, "step1")
step2 = Step2(AGENT_NAME, "step2")

#### Build the graph

In [5]:
class State(TypedDict):
    """
    The state of the agent
    """

    # inputs

    # the id of the request
    request_id: str

In [6]:
workflow = StateGraph(State)

# add nodes
workflow.add_node("Step1", step1)
workflow.add_node("Step2", step2)

# add edges
workflow.add_edge(START, "Step1")
workflow.add_edge("Step1", "Step2")
workflow.add_edge("Step2", END)

workflow_app = workflow.compile(checkpointer=checkpointer)

#### Execute

In [7]:
# set the input 
# it is fundamental for checkpointing pass a unique thread_id
request_id = "0005"

input_state: State = {
        "request_id": request_id
}

agent_config = {"configurable": {"thread_id": request_id}}

In [8]:
with zipkin_span(
            service_name=AGENT_NAME,
            span_name="run_agent",
            transport_handler=transporter.http_transport,
            encoding=Encoding.V2_JSON,
            sample_rate=100,
        ):
            result = workflow_app.invoke(input_state, config=agent_config)

Executed step1...
Executed step1...


#### After the execution we can see:
* 4 records have been registered in the checkpoint TABLE with thread_id = request_id
* the trace with two span (step1 and step2) is visible in OCI APM Trace Explorer